In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

In [3]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

(x, y), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x = x.astype("float32")
x_test = x_test.astype("float32")

In [7]:
x_train = x[:50000, ..., tf.newaxis]
y_train = y[:50000, ..., tf.newaxis]
x_valid = x[50000:, ..., tf.newaxis]
y_valid = y[50000:]
x_test = x_test[..., tf.newaxis]

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(128)
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(128)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(128)

In [9]:
train_ds

<DatasetV1Adapter shapes: ((?, 28, 28, 1), (?, 1)), types: (tf.float32, tf.uint8)>

In [11]:
class MNIST_model(Model):
    def __init__(self):
        super(MNIST_model, self).__init__() #öröklés
        self.conv1 = Conv2D(32, 5, activation = 'relu') #csak típus definíciók
        self.conv2 = Conv2D(64, 5, activation = 'relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation = 'relu')
        self.d2 = Dense(10, activation = 'softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MNIST_model()

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [13]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

valid_loss = tf.keras.metrics.Mean(name = 'valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'valid_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [21]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    predictions = model(images)
    v_loss = loss_object(labels, predictions)
    
    valid_loss(v_loss)
    valid_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [22]:
EPOCHS = 30
patience = 5
min_err = 99999
es_counter = 0

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
    for valid_images, valid_labels in valid_ds:
        valid_step(valid_images, valid_labels)
    
    template = 'Epoch {}, LOss: {}, Acc: {}, Val. loss: {}, val. acc: {}'
    print(template.format(epoch+1), train_loss.result(), train_accuracy.result(), valid_loss.result(), valid_accuracy.result())
    
    if(valid_loss.result().numpy() < min_err):
        min_err = valid_loss.result().numpy()
        es_counter = 0
    else:
        es_counter += 1
        
    if(es_counter > patience):
        print("Early stopping.\n")
        model.load_weights('weights.hdf5')
        
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

IndexError: tuple index out of range